Also hier geht's los

In [ ]:
#%% make sure figures appears inline and animations works
%matplotlib notebook

In [ ]:
__version__ = '0.1.0'

# import engine module
import sirf.Gadgetron as pMR
from sirf.Utilities import examples_data_path

# import further modules
import os, numpy as np

import matplotlib.pyplot as plt
import matplotlib.animation as animation

from tqdm.auto import trange

#%% GO TO MR FOLDER
os.chdir(examples_data_path('MR'))

In [ ]:
# This is just an auxiliary function
def norm_array( arr ):
    min_a = abs(arr).min()
    max_a = abs(arr).max()

    return (arr - min_a)/(max_a - min_a)
    

In [ ]:
# LOADING AND PREPROCESSING DATA FOR THIS SET
filename_grappa_file = 'ptb_resolutionphantom_GRAPPA4_ismrmrd.h5'
acq_data = pMR.AcquisitionData(filename_grappa_file)
preprocessed_data = pMR.preprocess_acquisition_data(acq_data)
preprocessed_data.sort()

In [ ]:
# WE DO A GRAPPA RECONSTRUCTION USING SIRF

recon = pMR.CartesianGRAPPAReconstructor()
recon.set_input(preprocessed_data)
recon.compute_gfactors(False)
print('---\n reconstructing...')

recon.process()
# for undersampled acquisition data GRAPPA computes Gfactor images
# in addition to reconstructed ones
grappa_images = recon.get_output()
grappa_images_array = grappa_images.as_array()
grappa_images_array = norm_array(grappa_images_array)

In [ ]:
csm = pMR.CoilSensitivityData()
csm.smoothness = 50
csm.calculate(preprocessed_data)


In [ ]:
# NOW WE GENERATE THE ACQUISITION MODEL
E = pMR.AcquisitionModel(preprocessed_data, grappa_images)
# to supply coil info to the acquisition model we use the dedicated method
E.set_coil_sensitivity_maps(csm)


In [ ]:
# Now we can hop back from k-space into image space in just one line:
aq_model_image = E.backward( preprocessed_data )
aq_model_image_array = norm_array(aq_model_image.as_array())

In [ ]:
# PLOT THE RESULTS
fig = plt.figure(figsize=(9, 4))
plt.set_cmap('gray')
ax = fig.add_subplot(1,2,1)
ax.imshow(abs(grappa_images_array[0,:,:]), vmin=0, vmax=1)
ax.set_title('GRAPPA')
ax.axis('off')

ax = fig.add_subplot(1,2,2)
ax.imshow(abs(aq_model_image_array[0,:,:]))
ax.set_title('Result Backward Method of E ')
ax.axis('off')
plt.tight_layout()




We want to use conjugate gradient routine of the `scipy.optimize.minimize`function to solve the equation $A x - y = 0$ where $A = A^H$

You need to supply the objective function and it's gradient as: `fun(x,*args) -> float`.

Also, `scipy` only acts on real data `x`, so we need to split our complex images into two channels: `[img] = [real(img), imag(img)]`

In [ ]:
import scipy.optimize as optimize

def split_complex_to_real(data):
    
    return np.concatenate( (np.real(data.flatten()), np.imag(data.flatten())))

def merge_real_to_complex(data):
    
    data = np.split(data,2)
    
    return data[0] + 1j*data[1]



In [ ]:
y = preprocessed_data
y = y / y.norm()

print(y.norm())

In [ ]:
def objective(x, *args):
    
    
    assert len(args)==3, "Please give three arguments only"
    E = args[0]
    i = args[1]
    y = args[2]
    
    data = merge_real_to_complex(x)
    i = i.fill(data)
    
    c =  E.forward(i)-y
    return c.norm() ** 2

def grad_objective(x, *args):
    
    assert len(args)==3, "Please give three arguments only"
    E = args[0]
    i = args[1]
    y = args[2]
    
    data = merge_real_to_complex(x)
    
    i = i.fill(data)    
    
    dObj = E.backward(E.forward(i) - y)
    dObj = 2*dObj.as_array().flatten()
    
    return split_complex_to_real(dObj)

In [ ]:
img = grappa_images
extra_args = (E, img, y)

cg_tol = 0
extra_options = {"maxiter": 150, "disp": True, "gtol":cg_tol}

Fix the starting conditions and check the cost function

In [ ]:
img.fill(0.0 + 1j*0.0)
x0 = split_complex_to_real(img.as_array().flatten().copy())
c0 = objective(x0, *extra_args)
print("Objective funtion start is: {}".format((c0)))


In [ ]:
# x0 = img.as_array().flatten()
res = optimize.minimize(objective, x0, args=extra_args, method='CG', jac=grad_objective, options=extra_options)
res = img.fill(res.x)
resarr = norm_array(res.as_array())

In [ ]:
# PLOT THE RESULTS
fig = plt.figure(figsize=(9, 4))
ax = fig.add_subplot(1,4,1)
ax.imshow(abs(grappa_images_array[0,:,:]), vmin=0, vmax=1)
ax.set_title('GRAPPA')
ax.axis('off')

plt.set_cmap('gray')
ax = fig.add_subplot(1,4,2)
ax.imshow(abs(aq_model_image_array[0,:,:]), vmin=0, vmax=1)
ax.set_title('Result Backward Method of E ')
ax.axis('off')
plt.tight_layout()

plt.set_cmap('gray')
ax = fig.add_subplot(1,4,3)
ax.imshow(abs(resarr[0,:,:]), vmin=0, vmax=1)
ax.set_title('Result CG Method')
ax.axis('off')
plt.tight_layout()

plt.set_cmap('gray')
ax = fig.add_subplot(1,4,4)
ax.imshow(abs(aq_model_image_array[0,:,:] - resarr[0,:,:]), vmin=0, vmax=0.5)
ax.set_title('Diff')
ax.axis('off')
plt.tight_layout()